In [18]:
import numpy as np
import pandas as pd

In [19]:
resultsDF = pd.read_csv("../../evaluation_kgrec_longTerm/results/result_raw", sep=",", header=0)
resultsDF.head()

,alg,group_id,user_id,metric,result
0,mf_230_1.0_avg20_sim_group_2,0,2747,AR,0.349839
1,mf_230_1.0_avg20_sim_group_2,0,2747,nDCG,0.651195
2,mf_230_1.0_avg20_sim_group_2,0,2047,AR,0.548269
3,mf_230_1.0_avg20_sim_group_2,0,2047,nDCG,0.755049
4,mf_230_1.0_avg20_sim_group_2,1,1244,AR,0.380399


In [20]:
resultsDF.shape

(1061500, 5)

### Show AVG result just to ensure that none of the algorithms went haywire

In [21]:
groupDF = resultsDF.groupby(["alg","group_id","user_id","metric"]).agg("mean")[["result"]]
groupDF.reset_index(inplace=True)
groupDF.head()

,alg,group_id,user_id,metric,result
0,mf_230_1.0_avg20_div_group_2,0,1062,AR,0.471498
1,mf_230_1.0_avg20_div_group_2,0,1062,nDCG,0.779219
2,mf_230_1.0_avg20_div_group_2,0,1415,AR,0.242903
3,mf_230_1.0_avg20_div_group_2,0,1415,nDCG,0.459933
4,mf_230_1.0_avg20_div_group_2,1,595,AR,0.230138


In [22]:
groupDF.shape

(212300, 5)

In [23]:
groupDF.to_csv("../../evaluation_kgrec_longTerm/results/res_group_folds.csv", sep=",", header=True)

In [24]:
from pathlib import Path
import sys, os
from typing import Dict, List, NamedTuple, Tuple
import numpy as np
from collections import defaultdict
import time


In [25]:
def compute_metrics(alg_name, groups, results):
    # test_data are triplets: user_id, item_id, and rating
    #LP: test data is matrix user_id x item_id !!!!!! a ja si rikal, jakto ze ti to prirazeni funguje...

    
    
    avg_rating = []
    min_rating = []
    minmax_rating = []
    std_rating = []
    
    avg_nDCG_rating = []
    min_nDCG_rating = []
    minmax_nDCG_rating = []
    std_nDCG_rating = []
        
    for gid in groups:
        gid_res_ndcg = results.loc[(results.group_id == gid)&(results.metric == "nDCG")]
        gid_res_AR = results.loc[(results.group_id == gid)&(results.metric == "AR")] 
        
        #print(gid_res_ndcg)
        #print(gid_res_AR)
        
        group_users_mean_ratings = [i for i in gid_res_AR.result]
        group_users_ndcg_ratings = [i for i in gid_res_ndcg.result]
        
        #print(group_users_mean_ratings)
        if len(group_users_mean_ratings) > 0:
            avg_rating.append(np.average(group_users_mean_ratings)) 
            min = np.min(group_users_mean_ratings)
            min_rating.append(min) 
            max = np.max(group_users_mean_ratings)
            minmax_rating.append(0.0 if max == 0.0 else min/max)
            std_rating.append(np.std(group_users_mean_ratings)) 

            avg_nDCG_rating.append(np.average(group_users_ndcg_ratings)) 
            min = np.min(group_users_ndcg_ratings)
            min_nDCG_rating.append(min) 
            max = np.max(group_users_ndcg_ratings)
            minmax_nDCG_rating.append(0.0 if max == 0.0 else min/max)
            std_nDCG_rating.append(np.std(group_users_ndcg_ratings)) 

    print(len(avg_rating))    
    results = pd.DataFrame({
       "AR_avg": avg_rating ,
        "AR_min": min_rating ,
        "AR_min/max": minmax_rating ,
        #"AR_std": std_rating ,
        
        "nDCG_avg": avg_nDCG_rating ,
        "nDCG_min": min_nDCG_rating ,
        "nDCG_min/max": minmax_nDCG_rating #,
        #"nDCG_std": std_nDCG_rating 
    })
      
    return results

In [26]:
algs = resultsDF.alg.unique()
groups = resultsDF.group_id.unique()
metrics = resultsDF.metric.unique()

In [27]:
metrics

array(['AR', 'nDCG'], dtype=object)

In [28]:
algs

array(['mf_230_1.0_avg20_sim_group_2',
       'mf_230_1.0_avg20_sim_group_2_weighted',
       'mf_230_1.0_avg_sim_group_2_AVG_longterm',
       'mf_230_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_1.0',
       'mf_230_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_1.0_weighted',
       'mf_230_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_longterm',
       'mf_230_1.0_avg_sim_group_2_FuzzyDHondt_longterm',
       'mf_230_1.0_avg_sim_group_2_GFAR_1.0',
       'mf_230_1.0_avg20_sim_group_3',
       'mf_230_1.0_avg_sim_group_3_AVG_longterm',
       'mf_230_1.0_avg_sim_group_3_FuzzyDHondtDirectOptimize_1.0',
       'mf_230_1.0_avg_sim_group_3_FuzzyDHondtDirectOptimize_longterm',
       'mf_230_1.0_avg_sim_group_3_FuzzyDHondt_longterm',
       'mf_230_1.0_avg_sim_group_3_GFAR_1.0',
       'mf_230_1.0_avg20_sim_group_8',
       'mf_230_1.0_avg_sim_group_8_AVG_longterm',
       'mf_230_1.0_avg_sim_group_8_FuzzyDHondtDirectOptimize_1.0',
       'mf_230_1.0_avg_sim_group_8_FuzzyDHondtDirectOpti

In [29]:
results = {}
for a in algs:
    if "weighted" not in a:
        resPerAlg = groupDF.loc[groupDF.alg == a]
        results[a] = compute_metrics(a, groups, resPerAlg)

355
355
355
355
355
355
355
355
355
355
355
355
335
335
335
335
335
335
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [30]:
res_mean = {}
for (key, val) in results.items():
    val = val.mean()
    res_mean[key] = val
res_meanDF = pd.DataFrame.from_dict(res_mean, orient="index")
res_meanDF.head()

,AR_avg,AR_min,AR_min/max,nDCG_avg,nDCG_min,nDCG_min/max
mf_230_1.0_avg20_sim_group_2,0.458781,0.400436,0.786103,0.740805,0.688689,0.871731
mf_230_1.0_avg_sim_group_2_AVG_longterm,0.452429,0.438820,0.942618,0.736177,0.711196,0.935133
mf_230_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_1.0,0.439821,0.437567,0.989907,0.721716,0.693018,0.925573
mf_230_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_longterm,0.439597,0.438837,0.996497,0.721475,0.691197,0.921742
mf_230_1.0_avg_sim_group_2_FuzzyDHondt_longterm,0.455070,0.444092,0.954082,0.740519,0.719681,0.945751


In [31]:
indices = [i for i in res_meanDF.index if "sim_group_2" in i]
res_meanDF.loc[indices]

,AR_avg,AR_min,AR_min/max,nDCG_avg,nDCG_min,nDCG_min/max
mf_230_1.0_avg20_sim_group_2,0.458781,0.400436,0.786103,0.740805,0.688689,0.871731
mf_230_1.0_avg_sim_group_2_AVG_longterm,0.452429,0.438820,0.942618,0.736177,0.711196,0.935133
mf_230_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_1.0,0.439821,0.437567,0.989907,0.721716,0.693018,0.925573
mf_230_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_longterm,0.439597,0.438837,0.996497,0.721475,0.691197,0.921742
mf_230_1.0_avg_sim_group_2_FuzzyDHondt_longterm,0.455070,0.444092,0.954082,0.740519,0.719681,0.945751
mf_230_1.0_avg_sim_group_2_GFAR_1.0,0.413003,0.393638,0.911720,0.674157,0.658890,0.956352


In [32]:
indices = [i for i in res_meanDF.index if "sim_group_3" in i]
res_meanDF.loc[indices]

,AR_avg,AR_min,AR_min/max,nDCG_avg,nDCG_min,nDCG_min/max
mf_230_1.0_avg20_sim_group_3,0.398875,0.338090,0.740774,0.634544,0.567129,0.811476
mf_230_1.0_avg_sim_group_3_AVG_longterm,0.393137,0.375140,0.916473,0.629320,0.592078,0.890293
mf_230_1.0_avg_sim_group_3_FuzzyDHondtDirectOptimize_1.0,0.385595,0.380827,0.976274,0.618341,0.586981,0.903079
mf_230_1.0_avg_sim_group_3_FuzzyDHondtDirectOptimize_longterm,0.385096,0.383614,0.992402,0.617938,0.584185,0.895617
mf_230_1.0_avg_sim_group_3_FuzzyDHondt_longterm,0.395080,0.379413,0.926626,0.632454,0.601579,0.908652
mf_230_1.0_avg_sim_group_3_GFAR_1.0,0.336758,0.312366,0.869279,0.550988,0.513666,0.877846


In [37]:
indices = [i for i in res_meanDF.index if ("sim_group_8" in i and "FuzzyDHondtDirectOptimize" in i)]
res_meanDF.loc[indices].round(3)

,AR_avg,AR_min,AR_min/max,nDCG_avg,nDCG_min,nDCG_min/max
mf_230_1.0_avg_sim_group_8_FuzzyDHondtDirectOptimize_1.0,0.280,0.258,0.871,0.438,0.387,0.799
mf_230_1.0_avg_sim_group_8_FuzzyDHondtDirectOptimize_longterm,0.279,0.267,0.932,0.436,0.389,0.803


In [34]:
indices = [i for i in res_meanDF.index if "div_group_2" in i]
res_meanDF.loc[indices]

,AR_avg,AR_min,AR_min/max,nDCG_avg,nDCG_min,nDCG_min/max
mf_230_1.0_avg20_div_group_2,0.356925,0.264613,0.619337,0.618873,0.496106,0.687211
mf_230_1.0_avg_div_group_2_AVG_longterm,0.337809,0.308691,0.844163,0.600776,0.560605,0.877141
mf_230_1.0_avg_div_group_2_FuzzyDHondtDirectOptimize_1.0,0.330477,0.327883,0.984387,0.592029,0.551819,0.877757
mf_230_1.0_avg_div_group_2_FuzzyDHondtDirectOptimize_longterm,0.330183,0.329331,0.994838,0.591869,0.549241,0.870988
mf_230_1.0_avg_div_group_2_FuzzyDHondt_longterm,0.349061,0.338606,0.941420,0.623325,0.584751,0.886257
mf_230_1.0_avg_div_group_2_GFAR_1.0,0.329201,0.302380,0.852466,0.583219,0.560751,0.926533


In [35]:
indices = [i for i in res_meanDF.index if "div_group_3" in i]
res_meanDF.loc[indices]

,AR_avg,AR_min,AR_min/max,nDCG_avg,nDCG_min,nDCG_min/max
mf_230_1.0_avg20_div_group_3,0.282684,0.186122,0.510176,0.490177,0.351273,0.579620
mf_230_1.0_avg_div_group_3_AVG_longterm,0.265932,0.242442,0.832624,0.473592,0.422724,0.807454
mf_230_1.0_avg_div_group_3_FuzzyDHondtDirectOptimize_1.0,0.260619,0.256667,0.970129,0.468364,0.424048,0.824994
mf_230_1.0_avg_div_group_3_FuzzyDHondtDirectOptimize_longterm,0.260286,0.259047,0.990128,0.468131,0.421976,0.817826
mf_230_1.0_avg_div_group_3_FuzzyDHondt_longterm,0.275263,0.261781,0.910686,0.491135,0.448824,0.840804
mf_230_1.0_avg_div_group_3_GFAR_1.0,0.246506,0.213456,0.771744,0.437409,0.398135,0.842552


In [38]:
indices = [i for i in res_meanDF.index if ("div_group_8" in i and "FuzzyDHondtDirectOptimize" in i)]
res_meanDF.loc[indices].round(3)

,AR_avg,AR_min,AR_min/max,nDCG_avg,nDCG_min,nDCG_min/max
mf_230_1.0_avg_div_group_8_FuzzyDHondtDirectOptimize_1.0,0.157,0.146,0.862,0.280,0.233,0.703
mf_230_1.0_avg_div_group_8_FuzzyDHondtDirectOptimize_longterm,0.156,0.152,0.942,0.279,0.233,0.701


In [39]:
from scipy import stats
#Comparison for size 8
def calculate_significance(results, grouptype, size, algs):
    print("")
    print(grouptype,size)

    otherAlgs = [a for a in algs if ( grouptype+"_group_"+size in a )]

    for count1, alg1 in enumerate(otherAlgs):
        baseDF = results[alg1]
        
        for count2,alg in enumerate(otherAlgs):
            if count1 != count2:
                otherDF = results[alg]
                res = {}
                for col in otherDF.columns:
                    mean_first = np.mean(baseDF.loc[:,col])
                    mean_second = np.mean(otherDF.loc[:,col])
                    mean_diff = np.mean(baseDF.loc[:,col] - otherDF.loc[:,col])
                    stat,pval = stats.ttest_rel(baseDF.loc[:,col], otherDF.loc[:,col])
                    if pval > 0.0001:
                        print("pval_excess",col)
                    res[col] = (mean_first,mean_second,mean_diff, pval)

                alg1 = alg1.replace("mf_230_1.0_avg_","")
                alg1 = alg1.replace(grouptype+"_group_"+size+"_","")
                alg = alg.replace("mf_230_1.0_avg_","")
                alg = alg.replace(grouptype+"_group_"+size+"_","")
                print(alg1+" - "+ alg)
                print(pd.DataFrame(res, index=["first","second","diff","pval"]))
                print("")

In [45]:
results_all = {}
algs_sign = []
whitelist = ['GFAR', 'AVG', 'FuzzyDHondt']
blacklist= "weighted"
for grouptype in ["sim","div"]:
    for size in ["2","3","8"]:
        resultsDF = groupDF.loc[groupDF.alg.str.contains(grouptype+"_group_"+size)]
        algs = resultsDF.alg.unique()        
        metrics = resultsDF.metric.unique()
        groups = resultsDF.group_id.unique()        
        results = {}
        for a in algs:
            for item in whitelist:
                if item in a and "weighted" not in a:
                    algs_sign.append(a)
                    resPerAlg = resultsDF.loc[resultsDF.alg == a]            
                    results[a] = compute_metrics(a, groups, resPerAlg)
        calculate_significance(results, grouptype, size, algs_sign)

355
355
355
355
355

sim 2
pval_excess AR_min
pval_excess nDCG_min/max
AVG_longterm - FuzzyDHondtDirectOptimize_1.0
               AR_avg    AR_min    AR_min/max       nDCG_avg      nDCG_min  \
first    4.524289e-01  0.438820  9.426179e-01   7.361767e-01  7.111962e-01   
second   4.398212e-01  0.437567  9.899065e-01   7.217165e-01  6.930180e-01   
diff     1.260776e-02  0.001254 -4.728861e-02   1.446024e-02  1.817823e-02   
pval    1.370255e-132  0.053634  2.185631e-65  3.678897e-111  8.590735e-53   

        nDCG_min/max  
first       0.935133  
second      0.925573  
diff        0.009560  
pval        0.000367  

pval_excess AR_min
AVG_longterm - FuzzyDHondtDirectOptimize_longterm
               AR_avg    AR_min    AR_min/max       nDCG_avg      nDCG_min  \
first    4.524289e-01  0.438820  9.426179e-01   7.361767e-01  7.111962e-01   
second   4.395969e-01  0.438837  9.964969e-01   7.214748e-01  6.911969e-01   
diff     1.283204e-02 -0.000017 -5.387903e-02   1.470186e-02  1.999938e-02

355
355
355
355
355

sim 3
AVG_longterm - FuzzyDHondtDirectOptimize_1.0
               AR_avg        AR_min    AR_min/max       nDCG_avg  nDCG_min  \
first    3.931371e-01  3.751398e-01  9.164731e-01   6.293205e-01  0.592078   
second   3.855947e-01  3.808266e-01  9.762740e-01   6.183410e-01  0.586981   
diff     7.542378e-03 -5.686847e-03 -5.980086e-02   1.097950e-02  0.005096   
pval    2.785308e-160  1.691160e-22  5.388641e-84  3.422923e-154  0.000003   

        nDCG_min/max  
first       0.890293  
second      0.903079  
diff       -0.012785  
pval        0.000010  

pval_excess nDCG_min/max
AVG_longterm - FuzzyDHondtDirectOptimize_longterm
               AR_avg        AR_min     AR_min/max       nDCG_avg  \
first    3.931371e-01  3.751398e-01   9.164731e-01   6.293205e-01   
second   3.850964e-01  3.836137e-01   9.924015e-01   6.179382e-01   
diff     8.040686e-03 -8.473967e-03  -7.592838e-02   1.138223e-02   
pval    3.757985e-140  2.438330e-37  6.898618e-100  1.961825e-162   



335
335
335
335
335

sim 8
pval_excess nDCG_min
AVG_longterm - FuzzyDHondtDirectOptimize_1.0
               AR_avg        AR_min    AR_min/max       nDCG_avg  nDCG_min  \
first    2.875336e-01  2.512039e-01  7.929228e-01   4.481961e-01  0.388041   
second   2.803520e-01  2.580181e-01  8.705292e-01   4.377672e-01  0.387408   
diff     7.181602e-03 -6.814190e-03 -7.760638e-02   1.042888e-02  0.000633   
pval    1.257205e-163  8.339727e-32  6.840753e-99  6.073875e-136  0.551050   

        nDCG_min/max  
first   7.662599e-01  
second  7.987153e-01  
diff   -3.245540e-02  
pval    1.127991e-21  

pval_excess nDCG_min
AVG_longterm - FuzzyDHondtDirectOptimize_longterm
               AR_avg        AR_min     AR_min/max       nDCG_avg  nDCG_min  \
first    2.875336e-01  2.512039e-01   7.929228e-01   4.481961e-01  0.388041   
second   2.786493e-01  2.672408e-01   9.322578e-01   4.359516e-01  0.389306   
diff     8.884365e-03 -1.603695e-02  -1.393350e-01   1.224454e-02 -0.001265   
pval    1.786

1000
1000
1000
1000
1000

div 2
pval_excess nDCG_min/max
AVG_longterm - FuzzyDHondtDirectOptimize_1.0
               AR_avg        AR_min     AR_min/max      nDCG_avg  \
first    3.378094e-01  3.086915e-01   8.441625e-01  6.007759e-01   
second   3.304769e-01  3.278834e-01   9.843867e-01  5.920291e-01   
diff     7.332510e-03 -1.919196e-02  -1.402242e-01  8.746829e-03   
pval    3.048202e-115  1.098783e-85  1.310418e-203  4.286626e-46   

            nDCG_min  nDCG_min/max  
first   5.606052e-01      0.877141  
second  5.518186e-01      0.877757  
diff    8.786579e-03     -0.000616  
pval    1.178069e-17      0.845740  

pval_excess nDCG_min/max
AVG_longterm - FuzzyDHondtDirectOptimize_longterm
               AR_avg        AR_min     AR_min/max      nDCG_avg  \
first    3.378094e-01  3.086915e-01   8.441625e-01  6.007759e-01   
second   3.301825e-01  3.293306e-01   9.948381e-01  5.918688e-01   
diff     7.626884e-03 -2.063915e-02  -1.506756e-01  8.907050e-03   
pval    1.987285e-124  1

1000
1000
1000
1000
1000

div 3
pval_excess nDCG_min
AVG_longterm - FuzzyDHondtDirectOptimize_1.0
               AR_avg         AR_min     AR_min/max      nDCG_avg  nDCG_min  \
first    2.659319e-01   2.424415e-01   8.326236e-01  4.735923e-01  0.422724   
second   2.606187e-01   2.566671e-01   9.701291e-01  4.683638e-01  0.424048   
diff     5.313197e-03  -1.422556e-02  -1.375055e-01  5.228489e-03 -0.001324   
pval    3.222409e-137  1.077439e-138  2.040356e-269  1.235028e-43  0.138207   

        nDCG_min/max  
first   8.074542e-01  
second  8.249943e-01  
diff   -1.754010e-02  
pval    1.905761e-09  

pval_excess nDCG_min
pval_excess nDCG_min/max
AVG_longterm - FuzzyDHondtDirectOptimize_longterm
               AR_avg         AR_min     AR_min/max      nDCG_avg  nDCG_min  \
first    2.659319e-01   2.424415e-01   8.326236e-01  4.735923e-01  0.422724   
second   2.602864e-01   2.590466e-01   9.901278e-01  4.681309e-01  0.421976   
diff     5.645519e-03  -1.660509e-02  -1.575043e-01  5.46

1000
1000
1000
1000
1000

div 8
AVG_longterm - FuzzyDHondtDirectOptimize_1.0
               AR_avg         AR_min     AR_min/max      nDCG_avg  \
first    1.600331e-01   1.359748e-01   7.392103e-01  2.834701e-01   
second   1.573078e-01   1.455993e-01   8.620208e-01  2.802585e-01   
diff     2.725280e-03  -9.624501e-03  -1.228105e-01  3.211605e-03   
pval    2.951929e-123  6.293752e-183  1.698800e-266  5.048080e-57   

            nDCG_min  nDCG_min/max  
first   2.256230e-01  6.578243e-01  
second  2.329764e-01  7.030267e-01  
diff   -7.353311e-03 -4.520240e-02  
pval    2.000485e-32  6.312208e-60  

AVG_longterm - FuzzyDHondtDirectOptimize_longterm
               AR_avg         AR_min  AR_min/max       nDCG_avg      nDCG_min  \
first    1.600331e-01   1.359748e-01     0.73921   2.834701e-01  2.256230e-01   
second   1.562553e-01   1.517197e-01     0.94156   2.791397e-01  2.329329e-01   
diff     3.777812e-03  -1.574492e-02    -0.20235   4.330375e-03 -7.309834e-03   
pval    1.844383e